In [1]:
import os
import random
import numpy as np
import cv2
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
from tqdm import tqdm

/opt/miniconda3/envs/courseml/lib/python3.10/site-packages/albumentations/check_version.py:147: UserWarning: Error fetching version info <urlopen error _ssl.c:1000: The handshake operation timed out>
  data = fetch_version_info()


In [2]:
DATA_DIR = './training/' 

IMAGE_DIR = os.path.join(DATA_DIR, 'images')
MASK_DIR = os.path.join(DATA_DIR, 'groundtruth')

# --- New settings for Patch-based Classification ---
DEVICE = 'cpu'
EPOCHS = 50
BATCH_SIZE = 64 # We can use a larger batch size now
LEARNING_RATE = 0.001
PATCH_SIZE = 16
FOREGROUND_THRESHOLD = 0.25 # From the project description

In [7]:
def get_training_augmentations():
    """
    Returns a set of heavy augmentations for the training data.
    """
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])

def get_validation_augmentations():
    """Returns minimal augmentations for validation (no resizing)."""
    return A.Compose([
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])

In [8]:
# --- Helper function to label a mask patch ---
def patch_to_label(patch, threshold):
    """
    Assigns a label (1 for road, 0 for background) to a mask patch
    based on the percentage of foreground pixels.
    """
    # Ground truth masks have values of 0 or 255.
    # We calculate the mean and normalize to a 0-1 range.
    foreground_percentage = np.mean(patch) / 255.0
    return 1 if foreground_percentage > threshold else 0

# --- New Dataset for 16x16 Patches ---
class PatchDataset(Dataset):
    def __init__(self, image_paths, mask_paths, augmentations=None):
        self.augmentations = augmentations
        self.patches = []

        print("Creating patches... This might take a moment.")
        # Iterate through images and create patches
        for img_path, mask_path in tqdm(zip(image_paths, mask_paths), total=len(image_paths)):
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            
            img_h, img_w, _ = image.shape
            
            for y in range(0, img_h, PATCH_SIZE):
                for x in range(0, img_w, PATCH_SIZE):
                    # Ensure the patch is fully within the image bounds
                    if y + PATCH_SIZE > img_h or x + PATCH_SIZE > img_w:
                        continue
                        
                    image_patch = image[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
                    mask_patch = mask[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
                    
                    label = patch_to_label(mask_patch, FOREGROUND_THRESHOLD)
                    
                    self.patches.append((image_patch, label))

    def __len__(self):
        return len(self.patches)

    def __getitem__(self, idx):
        image_patch, label = self.patches[idx]
        
        if self.augmentations:
            augmented = self.augmentations(image=image_patch)
            image_patch = augmented['image']
            
        # Return the patch and its label as a tensor
        return image_patch, torch.tensor([label], dtype=torch.float32)

In [9]:
# --- New CNN Model for Patch Classification ---
class PatchClassifier(nn.Module):
    def __init__(self, in_channels=3, out_features=1):
        super(PatchClassifier, self).__init__()
        self.network = nn.Sequential(
            # Input: 3 x 16 x 16
            nn.Conv2d(in_channels, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # -> 32 x 8 x 8
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # -> 64 x 4 x 4
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # -> 128 x 2 x 2
            
            nn.Flatten(),
            nn.Linear(128 * 2 * 2, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, out_features),
            nn.Sigmoid() # To output a probability (0 to 1)
        )
        
    def forward(self, x):
        return self.network(x)

# --- New F1 Score for Classification ---
def f1_score_classification(outputs, labels, threshold=0.5):
    # Convert model outputs (probabilities) to binary predictions (0 or 1)
    preds = (outputs > threshold).float()
    
    tp = (preds * labels).sum()
    fp = (preds * (1 - labels)).sum()
    fn = ((1 - preds) * labels).sum()
    
    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)
    
    f1 = 2 * (precision * recall) / (precision + recall + 1e-6)
    return f1.item()


# --- Model, Loss, and Optimizer Setup ---
model = PatchClassifier().to(DEVICE)
loss_fn = nn.BCELoss() # Binary Cross Entropy Loss for classification
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [15]:
# --- Data Splitting and DataLoader Setup ---
all_image_ids = sorted(os.listdir(IMAGE_DIR))
random.seed(42)
random.shuffle(all_image_ids)

# Split into train and validation sets (80/20)
train_size = int(0.8 * len(all_image_ids))
train_ids = all_image_ids[:train_size]
valid_ids = all_image_ids[train_size:]

# Create lists of full file paths for the datasets
train_image_paths = [os.path.join(IMAGE_DIR, img_id) for img_id in train_ids]
train_mask_paths = [os.path.join(MASK_DIR, img_id) for img_id in train_ids]

valid_image_paths = [os.path.join(IMAGE_DIR, img_id) for img_id in valid_ids]
valid_mask_paths = [os.path.join(MASK_DIR, img_id) for img_id in valid_ids]

# Create Dataset and DataLoader instances
train_dataset = PatchDataset(train_image_paths, train_mask_paths, augmentations=get_training_augmentations())
valid_dataset = PatchDataset(valid_image_paths, valid_mask_paths, augmentations=get_validation_augmentations())

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"\nCreated {len(train_dataset)} training patches and {len(valid_dataset)} validation patches.")

Creating patches... This might take a moment.


100%|██████████| 80/80 [00:00<00:00, 185.31it/s]


Creating patches... This might take a moment.


100%|██████████| 20/20 [00:00<00:00, 185.05it/s]


Created 50000 training patches and 12500 validation patches.


In [17]:
# --- PyTorch Training and Validation Loop ---

max_fscore = 0
best_model_path = 'best_model.pth'

for epoch in range(EPOCHS):
    print(f'\nEpoch: {epoch + 1}/{EPOCHS}')
    
    # --- Training Phase ---
    model.train()
    train_loss, train_fscore, train_correct = 0, 0, 0
    train_pbar = tqdm(train_loader, desc=f"Training", leave=False)
    
    for patches, labels in train_pbar:
        patches, labels = patches.to(DEVICE), labels.to(DEVICE)
        
        # Forward pass
        outputs = model(patches)
        loss = loss_fn(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Accumulate metrics
        train_loss += loss.item()
        f1 = f1_score_classification(outputs, labels)
        train_fscore += f1
        
        # Update progress bar
        train_pbar.set_postfix(loss=loss.item(), f1=f1)
    
    # --- Validation Phase ---
    model.eval()
    val_loss, val_fscore = 0, 0
    with torch.no_grad():
        val_pbar = tqdm(valid_loader, desc=f"Validation", leave=False)
        for patches, labels in val_pbar:
            patches, labels = patches.to(DEVICE), labels.to(DEVICE)
            
            # Forward pass
            outputs = model(patches)
            loss = loss_fn(outputs, labels)
            
            # Accumulate metrics
            val_loss += loss.item()
            f1 = f1_score_classification(outputs, labels)
            val_fscore += f1
            
            # Update progress bar
            val_pbar.set_postfix(loss=loss.item(), f1=f1)

    # --- Logging and Model Saving ---
    avg_train_loss = train_loss / len(train_loader)
    avg_train_fscore = train_fscore / len(train_loader)
    
    avg_val_loss = val_loss / len(valid_loader)
    avg_val_fscore = val_fscore / len(valid_loader)
    
    print(f"Train -> Loss: {avg_train_loss:.4f}, F1-Score: {avg_train_fscore:.4f}")
    print(f"Valid -> Loss: {avg_val_loss:.4f}, F1-Score: {avg_val_fscore:.4f}")
    
    # Save the model if it has the best validation F1-score so far
    if avg_val_fscore > max_fscore:
        max_fscore = avg_val_fscore
        torch.save(model.state_dict(), best_model_path)
        print(f"Model saved! Best Validation F1-Score: {max_fscore:.4f}")


Epoch: 1/50


Train -> Loss: 0.4891, F1-Score: 0.3338
Valid -> Loss: 0.4381, F1-Score: 0.4234
Model saved! Best Validation F1-Score: 0.4234

Epoch: 2/50


Train -> Loss: 0.4182, F1-Score: 0.5793
Valid -> Loss: 0.4146, F1-Score: 0.4517
Model saved! Best Validation F1-Score: 0.4517

Epoch: 3/50


Train -> Loss: 0.4014, F1-Score: 0.6094
Valid -> Loss: 0.4275, F1-Score: 0.5179
Model saved! Best Validation F1-Score: 0.5179

Epoch: 4/50


Train -> Loss: 0.3885, F1-Score: 0.6216
Valid -> Loss: 0.4031, F1-Score: 0.4759

Epoch: 5/50


...

Train -> Loss: 0.2562, F1-Score: 0.7666
Valid -> Loss: 0.4405, F1-Score: 0.5570

Epoch: 47/50


Train -> Loss: 0.2542, F1-Score: 0.7687
Valid -> Loss: 0.3963, F1-Score: 0.5339

Epoch: 48/50


Train -> Loss: 0.2583, F1-Score: 0.7649
Valid -> Loss: 0.3987, F1-Score: 0.5506

Epoch: 49/50


Train -> Loss: 0.2539, F1-Score: 0.7675
Valid -> Loss: 0.3854, F1-Score: 0.5521

Epoch: 50/50


Train -> Loss: 0.2515, F1-Score: 0.7729
Valid -> Loss: 0.3987, F1-Score: 0.5583


In [18]:
# --- Final Prediction and Submission ---
import re

TEST_IMAGE_DIR = './test_set_images/'
SUBMISSION_FILENAME = 'submission.csv'

# Load the best trained model
print("\nLoading best patch classifier model for prediction...")
best_model = PatchClassifier().to(DEVICE)
best_model.load_state_dict(torch.load('best_model.pth'))
best_model.eval()
print("Model loaded.")

# Get augmentations for test data
test_augmentations = get_validation_augmentations()

# Collect all test image file paths
test_image_paths = []
for root, _, files in os.walk(TEST_IMAGE_DIR):
    for file in files:
        if file.endswith(".png"):
            test_image_paths.append(os.path.join(root, file))
test_image_paths.sort()

print(f"Found {len(test_image_paths)} test images. Generating submission file...")
with open(SUBMISSION_FILENAME, "w") as f:
    f.write("id,prediction\n")
    
    for img_path in tqdm(test_image_paths):
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        img_h, img_w, _ = image.shape
        img_id_str = os.path.basename(img_path)
        img_number = int(re.search(r"(\d+)", img_id_str).group(1))
        
        for y in range(0, img_h, PATCH_SIZE):
            for x in range(0, img_w, PATCH_SIZE):
                if y + PATCH_SIZE > img_h or x + PATCH_SIZE > img_w:
                    continue
                
                image_patch = image[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
                
                # Apply transformations
                augmented = test_augmentations(image=image_patch)
                input_tensor = augmented['image'].unsqueeze(0).to(DEVICE)

                # Predict with the model
                with torch.no_grad():
                    output = best_model(input_tensor)
                    prediction = 1 if output.item() > 0.5 else 0
                
                # Write to submission file
                submission_id = "{:03d}_{}_{}".format(img_number, x, y)
                f.write(f"{submission_id},{prediction}\n")

print(f"\n--- Submission file '{SUBMISSION_FILENAME}' created successfully! ---")


Loading best patch classifier model for prediction...
Model loaded.
Found 50 test images. Generating submission file...


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:14<00:00,  3.43it/s]


--- Submission file 'submission.csv' created successfully! ---
